In [1]:
import math

In [2]:
fuelDensityDict = {"LOX": 1140, "LH2": 71, "RP1": 820, "N2O4": 1450, "UDMH": 793}

tankRegressionList = [{"x": 12.158, "y": 0.9328, "chemical_compound": ["LOX", "RP1", "N2O4", "UDMH"]},
                      {"x": 9.0911, "y": 0.9896, "chemical_compound": ["LH2"]}]

In [6]:
class Structure():
    def __init__(self,
                 oxName,
                 fuelName,
                 totalFuelMass,
                 MR,
                 deltaV,
                 Isp,
                 delta,
                 V_exhaust,
                 payloadMass = 0,
                 tankFormat = "spherical",
                 radius=2,
                 coneHeight=None,
                 bottomCylinderHeight=None):
        self.oxName = oxName
        self.fuelName = fuelName
        self.totalFuelMass = totalFuelMass
        self.MR = MR
        self.tankFormat = tankFormat
        self.radius=radius
        self.coneHeight = coneHeight
        self.bottomCylinderHeight = bottomCylinderHeight

        self.deltaV = deltaV
        self.Isp = Isp
        self.delta = delta
        self.V_exhaust = V_exhaust

        self.payloadMass=payloadMass

        self.oxMass = MR * totalFuelMass / (1 + MR)
        self.fuelMass = totalFuelMass / (1 + MR)
        print(f"Ox Mass: {self.oxMass}")
        print(f"Fuel Mass: {self.fuelMass}")

        self.oxTankVolume = None
        self.fuelTankVolume = None
        self.oxTankMass = None
        self.fuelTankMass = None

    def calculate_design_parameters(self):
        self.r = math.exp(- (self.deltaV/self.V_exhaust))
        self.lambda_value = self.r - self.delta
        self.M_0 = self.payloadMass/ self.lambda_value
        self.M_i = self.delta * self.M_0
        self.M_p = self.M_0 * (1 - self.r)
        print(self.r, self.lambda_value, self.M_0, self.M_i, self.M_p)


    def estimate_tank_volume(self):
        self.oxTankVolume = self.oxMass / fuelDensityDict[self.oxName] 
        self.fuelTankVolume = self.fuelMass / fuelDensityDict[self.fuelName] 


    def estimate_tank_mass(self):
        for regression in tankRegressionList:
            if self.oxName in regression["chemical_compound"]:
                self.oxTankMass = self.oxTankVolume * regression["x"] + regression ["y"]
                break
        for regression in tankRegressionList:
            if self.fuelName in regression["chemical_compound"]:
                self.fuelTankMass = self.fuelTankVolume * regression["x"] + regression ["y"]
                break

        print(f"Ox Tank Mass: {self.oxTankMass}")
        print(f"Fuel Tank Mass: {self.fuelTankMass}")

    def estimate_tank_surface_area(self):
        if self.tankFormat == "spherical":
            self.oxTankRadius = (self.oxTankVolume / (4 * math.pi / 3)) ** (1/3)
            self.oxTankSurfaceArea = 4 * math.pi * (self.oxTankRadius ** 2)
            self.fuelTankRadius = (self.fuelTankVolume / (4 * math.pi / 3)) ** (1/3)
            self.fuelTankSurfaceArea = 4 * math.pi * (self.fuelTankRadius ** 2)
            print(f"Ox Tank Radius: {self.oxTankRadius}")
            print(f"Fuel Tank Radius: {self.fuelTankRadius}")
        
        if self.tankFormat == "cylyndrical":
            self.oxTankEsfVolume = (4/3) * math.pi * (self.radius ** 3)
            self.fuelTankEsfVolume = (4/3) * math.pi * (self.radius ** 3)
            self.oxTankCylVolume = self.oxTankVolume - self.oxTankEsfVolume
            self.fuelTankCylVolume = self.fuelTankVolume - self.fuelTankEsfVolume
            self.oxTankCylHeight = self.oxTankCylVolume/ (math.pi * (self.radius ** 2))
            self.fuelTankCylHeight = self.fuelTankCylVolume/ (math.pi * (self.radius ** 2))
            print(self.oxTankEsfVolume, self.fuelTankEsfVolume)
            print(self.oxTankCylVolume, self.fuelTankCylVolume)

            print(self.oxTankCylHeight,self.fuelTankCylHeight)

            self.oxTankSurfaceArea = 4 * math.pi * (self.radius ** 2) + 2 * math.pi * self.radius * self.oxTankCylHeight
            self.fuelTankSurfaceArea = 4 * math.pi * (self.radius ** 2) + 2 * math.pi * self.radius * self.fuelTankCylHeight

        print(f"Ox Tank Surface Area: {self.oxTankSurfaceArea}")
        print(f"Fuel Tank Surface Area: {self.fuelTankSurfaceArea}")
    
    def estimate_cryogenic_insulation_mass(self):
        insulationFactor = {"LH2": 2.88, "LOX": 1.123, "RP1": 0}
        self.oxTankInsulationMass = insulationFactor[self.oxName] * self.oxTankSurfaceArea
        self.fuelTankInsulationMass = insulationFactor[self.fuelName] * self.fuelTankSurfaceArea
        print(f"Ox Tank Insulation: {self.oxTankInsulationMass}")
        print(f"Fuel Tank Insulation: {self.fuelTankInsulationMass}")

    def estimate_fairing_surface_area(self):
        if self.tankFormat == "spherical":
            minRadius = min(self.oxTankRadius, self.fuelTankRadius)
            maxRadius = max(self.oxTankRadius, self.fuelTankRadius)
            coneArea = math.pi * minRadius * math.sqrt((minRadius ** 2) + (self.coneHeight ** 2))
            # Verificar frustrum
            frustrumArea = math.pi * (minRadius + maxRadius) * math.sqrt((maxRadius - minRadius) ** 2 + self.coneHeight ** 2)
            cylinderArea = math.pi * 2 * maxRadius * self.bottomCylinderHeight
            self.totalFairingArea = coneArea + frustrumArea + cylinderArea
            self.payloadFairingArea = coneArea
            self.frustrumFairingArea = frustrumArea
            self.cylinderFairingArea = cylinderArea

            self.length = self.coneHeight + maxRadius + minRadius + self.bottomCylinderHeight

            self.fairingAreaList = [coneArea, frustrumArea, cylinderArea]
            print(f"Payload Fairing Area: {coneArea}")
            print(f"Intertank Fairing Area: {frustrumArea}")
            print(f"Aft Fairing Area: {cylinderArea}")
        
        if self.tankFormat == "cylyndrical":
            ...


        print(f"Total Fairing Area: {self.totalFairingArea}")
    
    def estimate_fairing_mass(self):
        self.totalFairingMass = 4.95 * (self.totalFairingArea) ** 1.15
        self.frustrumFairingMass = 4.95 * (self.frustrumFairingArea) ** 1.15
        self.cylinderFairingMass = 4.95 * (self.cylinderFairingArea) ** 1.15
        self.payloadFairingMass = 4.95 * (self.payloadFairingArea) ** 1.15

        self.totalFairingMass = 0
        for fairingArea in self.fairingAreaList:
            self.totalFairingMass += 4.95 * (fairingArea) ** 1.15
        print(f"Total Fairing Mass: {self.totalFairingMass}")
    
    def estimate_avionics_mass(self):
        self.avionicsMass = 10 * (self.M_0 ** 0.361)
        print(f"Avionics Mass: {self.avionicsMass}")
        return 
    
    def estimate_wiring_mass(self):
        self.wiringMass = 1.058 * math.sqrt(self.M_0) * (self.length ** 0.25)
        print(f"Wiring Mass: {self.wiringMass}")

        return 
    
    def estimate_all(self):
        self.calculate_design_parameters()
        self.estimate_tank_volume()
        self.estimate_tank_mass()
        self.estimate_tank_surface_area()
        self.estimate_cryogenic_insulation_mass()
        self.estimate_fairing_surface_area()
        self.estimate_fairing_mass()
        self.estimate_avionics_mass()
        self.estimate_wiring_mass()
                

totalFuelMass = 19390 + 116400
structure = Structure(oxName="LOX",
                      fuelName="LH2",
                      totalFuelMass=totalFuelMass,
                      MR=6,
                      coneHeight=7,
                      bottomCylinderHeight=7,
                      deltaV=9200,
                      payloadMass=5000,
                      Isp=430,
                      V_exhaust=4214,
                      delta=0.08323,
                      radius=2.1,
                      tankFormat="cylyndrical",
                      )
structure.estimate_all()

Ox Mass: 116391.42857142857
Fuel Mass: 19398.571428571428
0.11268050424798647 0.029450504247986467 169776.3799864938 14130.488106275878 150645.8918802179
Ox Tank Mass: 1242.2371759398495
Fuel Tank Mass: 2484.8537227364186
38.79238608652677 38.79238608652677
63.305358274375486 234.42692980884547
4.569324577582892 16.92072774282746
Ox Tank Surface Area: 115.70851181349107
Fuel Tank Surface Area: 278.6814370844149
Ox Tank Insulation: 129.94065876655048
Fuel Tank Insulation: 802.6025388031148


AttributeError: 'Structure' object has no attribute 'totalFairingArea'

In [ ]:
totalFuelMass = 19390 + 116400
structure = Structure(oxName="LOX",
                      fuelName="LH2",
                      totalFuelMass=totalFuelMass,
                      MR=6,
                      coneHeight=7,
                      bottomCylinderHeight=7,
                      deltaV=9200,
                      payloadMass=5000,
                      Isp=430,
                      V_exhaust=4214,
                      delta=0.08,
                      radius=4.0,
                      tankFormat="spherical",
                      )
structure.estimate_all()

Ox Mass: 116391.42857142857
Fuel Mass: 19398.571428571428
0.11268050424798647 0.032680504247986464 152996.41529576655 12239.713223661325 135756.70207210522
Ox Tank Mass: 1242.2371759398495
Fuel Tank Mass: 2484.8537227364186
Ox Tank Radius: 2.8994069750677927
Fuel Tank Radius: 4.025386601949033
Ox Tank Surface Area: 105.6399586938102
Fuel Tank Surface Area: 203.6221681885786
Ox Tank Insulation: 118.63367361314886
Fuel Tank Insulation: 586.4318443831063
Payload Fairing Area: 69.01441589977713
Intertank Fairing Area: 154.24168961211464
Aft Fairing Area: 177.04574967158607
Total Fairing Area: 400.30185518347787
Total Fairing Mass: 4175.467688244482
Avionics Mass: 744.1661875127884
Wiring Mass: 885.0984720079764
